In [103]:
import pandas as pd

# Load the Excel file into a DataFrame
df_bridges_all = pd.read_excel("bridges_data_p2.xlsx")

In [104]:
import geopandas as gpd
# Replace the file path with your actual file path
region_path = r'C:\Users\Doktor\Documents\GitHub\Bridges\GeoJSONs\regions_epsg_4326.geojson'  # doma
# region_path = r'C:\Users\relia\Documents\GitHub\Bridges\GeoJSONs\regions_epsg_4326.geojson'  # robota

# Load the GeoJSON file into a GeoDataFrame
region_gdf = gpd.read_file(region_path)


districts_path = r'C:\Users\Doktor\Documents\GitHub\Bridges\GeoJSONs\districts_epsg_4326.geojson'  # doma
# districts_path = r'C:\Users\relia\Documents\GitHub\Bridges\GeoJSONs\districts_epsg_4326.geojson'  # robota

# Load the GeoJSON file into a GeoDataFrame
districts_gdf = gpd.read_file(districts_path)


# Replace the file path with your actual file path
file_path = r'C:\Users\Doktor\Documents\GitHub\Bridges\GeoJSONs\cities_regions_districts_epsg_4326.geojson'  # doma
# file_path = r'C:\Users\relia\Documents\GitHub\Bridges\GeoJSONs\cities_regions_districts_epsg_4326.geojson'  # robota


# Load the GeoJSON file into a GeoDataFrame
settle_gdf = gpd.read_file(file_path)

# Define the types to exclude
types_to_exclude = ['country', 'airport', 'Psie skaly',
                    'county', 'Drevenný kostol', 'Vodopád']

# Create a boolean mask to select rows where the 'type' column is not in types_to_exclude
# and the 'population' column is not null
mask = (~settle_gdf['type'].isin(types_to_exclude)) & (
    settle_gdf['population'] > 0)


# Apply the mask to filter the GeoDataFrame in place
settle_gdf = settle_gdf[mask]

# Drop rows where the 'name' column is "Bratislava" or "Košice" from settle_gdf
settle_gdf = settle_gdf[~settle_gdf["name"].isin(["Bratislava", "Košice"])]

In [105]:
import numpy as np

# Define a function to select a random bridge from each group


def random_bridge(group):
    # Select one random sample from each group
    return group.sample(5, random_state=420)


# Group by "Kraj" and apply the random_bridge function
random_bridges = df_bridges_all.groupby(
    "Kraj", group_keys=False).apply(random_bridge)

# Reset index to make it a DataFrame
random_bridges = random_bridges.reset_index(drop=True)

# Display the randomly selected bridges
random_bridges.head()

,ID_mosta,Trieda_PK,Číslo_PK,Názov_mosta,správcovské_číslo,Zemepisná_dĺžka,Zemepisná_šírka,Výška,Druh_konštrukcie,Material,...,n_2016,n_2017,n_2018,n_2019,n_2020,n_2021,n_2022,n_2023,Trieda_PK_n,Okres_n
0,M35,cesta III. triedy,2749,2749-1 (050126-001) Most cez potok Pápča v obc...,1,20.064702,48.468025,227.524500,dosková,monolitický železobetón,...,2,2,2,2,2,4,4,4,4,609
1,M925,cesta I. triedy,65,"65_035 Most cez Záhrbský potok, k.ú. Nová Baňa",35,18.633559,48.409204,203.823999,dosková,monolitický železobetón,...,3,3,3,3,3,4,4,5,2,612
2,M5982,cesta III. triedy,2418,2418-2_Most cez Selčiansky potok v B. Bystrici...,2,19.187009,48.746536,361.146999,dosková,monolitický železobetón,...,3,3,3,3,3,3,3,4,4,601
3,M9611,diaľnica,R2,Most na R2 nad Slatinou,116,19.363950,48.540930,362.000000,dosková,monolitický predpätý betón,...,0,0,0,0,0,0,2,2,1,604
4,M1069,cesta I. triedy,75,"75_057 Most cez zátopové územie, k.ú. Pôtor",57,19.417761,48.230906,201.504499,dosková,monolitický železobetón,...,3,3,3,6,6,6,6,6,2,610


In [107]:
import pandas as pd
from geopy.distance import geodesic


def calculate_distances(settlements_gdf, bridges_df):
    # Create a list of column names with 'osm_id' and 'name' as the first two columns
    # Using bridge IDs directly
    column_names = ['name'] + bridges_df['ID_mosta'].tolist()

    # Initialize an empty DataFrame to hold distances with 'osm_id' as the index
    distances_df = pd.DataFrame(
        columns=column_names, index=settlements_gdf['osm_id'])

    # Fill the 'name' column with settlement names
    distances_df['name'] = settlements_gdf.set_index('osm_id')['name']

    total_settlements = len(settlements_gdf)

    # Iterate over each settlement
    for i, settle_row in settlements_gdf.iterrows():
        if i % 10 == 0:  # Progress tracking
            print(f"Processing settlement {i + 1} of {total_settlements}...")

        # Extract the settlement point
        settle_point = (settle_row['geometry'].y, settle_row['geometry'].x)

        # Calculate distances to each bridge
        for _, bridge_row in bridges_df.iterrows():
            bridge_point = (bridge_row['Zemepisná_šírka'],
                            bridge_row['Zemepisná_dĺžka'])
            distance_km = geodesic(settle_point, bridge_point).kilometers
            # Use osm_id as index and bridge ID directly for the column name
            distances_df.at[settle_row['osm_id'],
                            bridge_row['ID_mosta']] = distance_km

    print("Completed processing all settlements.")
    return distances_df

In [108]:
# Assuming `settle_gdf` and `df_bridges_all` are your GeoDataFrames for settlements and bridges, respectively
distance_matrix_df = calculate_distances(settle_gdf, df_bridges_all)

Processing settlement 21 of 2790...
Processing settlement 31 of 2790...
Processing settlement 41 of 2790...
Processing settlement 51 of 2790...
Processing settlement 61 of 2790...
Processing settlement 71 of 2790...
Processing settlement 81 of 2790...
Processing settlement 91 of 2790...
Processing settlement 101 of 2790...
Processing settlement 111 of 2790...
Processing settlement 121 of 2790...
Processing settlement 131 of 2790...
Processing settlement 141 of 2790...
Processing settlement 151 of 2790...
Processing settlement 161 of 2790...
Processing settlement 171 of 2790...
Processing settlement 181 of 2790...
Processing settlement 191 of 2790...
Processing settlement 201 of 2790...
Processing settlement 211 of 2790...
Processing settlement 221 of 2790...
Processing settlement 231 of 2790...
Processing settlement 241 of 2790...
Processing settlement 251 of 2790...
Processing settlement 261 of 2790...
Processing settlement 271 of 2790...
Processing settlement 281 of 2790...
Processin

In [110]:
distance_matrix_df.head()

,name,M5897,M1979,M6525,M4584,M3053,M1833,M1055,M1839,M4986,...,M10157,M10121,M10150,M10149,M10080,M9963,M9965,M9966,M9967,M9969
osm_id,,,,,,,,,,,,,,,,,,,,,
15243569,Hronský Beňadik,116.976399,118.516533,119.026222,119.119095,119.708158,120.057282,119.068176,119.103742,118.567045,...,210.534334,122.342586,163.449276,179.095084,298.016698,73.436079,74.292635,75.108505,76.195664,76.624811
26035645,Beluša,93.845493,98.161689,104.510462,106.229239,108.677732,109.336695,111.026233,111.903995,112.708207,...,198.415978,133.44751,161.951112,178.840621,302.837417,110.05927,111.538493,112.433317,115.204495,116.415289
26035646,Pusté Pole,280.337858,284.250451,289.411728,290.745991,292.755179,293.352678,294.241404,294.874676,295.24005,...,13.834016,77.689447,35.181937,24.967998,116.988018,129.917968,129.783758,129.323338,129.945066,130.34435
26035647,Hažín nad Cirochou,355.305053,358.838306,363.090334,364.158289,365.861759,366.410263,366.823692,367.308218,367.42954,...,79.916078,138.321136,103.972595,87.123382,39.113537,185.779693,185.067254,184.318319,183.695786,183.530443
26035648,Demandice,143.128855,144.16528,143.649719,143.436846,143.635869,143.904001,142.43412,142.292533,141.505549,...,211.163158,120.031923,162.531213,176.822303,290.249361,67.79927,68.09933,68.66743,68.580717,68.483766


In [111]:
distance_matrix_df.columns

Index(['name', 'M5897', 'M1979', 'M6525', 'M4584', 'M3053', 'M1833', 'M1055',
       'M1839', 'M4986',
       ...
       'M10157', 'M10121', 'M10150', 'M10149', 'M10080', 'M9963', 'M9965',
       'M9966', 'M9967', 'M9969'],
      dtype='object', length=8126)

In [113]:
# Convert all columns to float, ignoring errors for columns that cannot be converted
distance_matrix_df = distance_matrix_df.apply(pd.to_numeric, errors='ignore')

In [114]:
# Round the entire DataFrame to one decimal place
rounded_df = distance_matrix_df.round(1)
rounded_df

,name,M5897,M1979,M6525,M4584,M3053,M1833,M1055,M1839,M4986,...,M10157,M10121,M10150,M10149,M10080,M9963,M9965,M9966,M9967,M9969
osm_id,,,,,,,,,,,,,,,,,,,,,
15243569,Hronský Beňadik,117.0,118.5,119.0,119.1,119.7,120.1,119.1,119.1,118.6,...,210.5,122.3,163.4,179.1,298.0,73.4,74.3,75.1,76.2,76.6
26035645,Beluša,93.8,98.2,104.5,106.2,108.7,109.3,111.0,111.9,112.7,...,198.4,133.4,162.0,178.8,302.8,110.1,111.5,112.4,115.2,116.4
26035646,Pusté Pole,280.3,284.3,289.4,290.7,292.8,293.4,294.2,294.9,295.2,...,13.8,77.7,35.2,25.0,117.0,129.9,129.8,129.3,129.9,130.3
26035647,Hažín nad Cirochou,355.3,358.8,363.1,364.2,365.9,366.4,366.8,367.3,367.4,...,79.9,138.3,104.0,87.1,39.1,185.8,185.1,184.3,183.7,183.5
26035648,Demandice,143.1,144.2,143.6,143.4,143.6,143.9,142.4,142.3,141.5,...,211.2,120.0,162.5,176.8,290.2,67.8,68.1,68.7,68.6,68.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418724594,Čabiny,352.3,356.1,361.0,362.2,364.1,364.7,365.4,366.0,366.2,...,64.9,140.3,101.0,84.3,45.5,190.8,190.3,189.7,189.5,189.5
418724596,Vladiča,345.2,349.2,354.4,355.7,357.7,358.3,359.2,359.8,360.2,...,54.4,137.9,96.6,80.9,60.6,189.6,189.2,188.6,188.8,188.9
418724597,Korunková,341.3,345.2,350.1,351.3,353.3,353.8,354.6,355.2,355.5,...,53.5,130.4,90.4,73.9,56.6,181.4,180.9,180.3,180.2,180.3


In [115]:
# Assuming 'distance_matrix_df' is your DataFrame
rounded_df.to_csv('distances3.csv', float_format='%.1f')